In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import *
from statsmodels.graphics.tsaplots import *
from catboost import *
from sklearn.model_selection import *
from sklearn.metrics import *
from sklearn.datasets import load_diabetes
from sklearn.model_selection import *
from sklearn.linear_model import *
from sklearn.preprocessing import PolynomialFeatures
from sklearn.tree import *
from sklearn.ensemble import *
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.svm import *
from sklearn.neighbors import *
from sklearn.neural_network import *
from sklearn.metrics import *
import pickle
import tensorflow as tf
import shap
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.graphics.tsaplots import *
import os

# tf.compat.v1.disable_v2_behavior()
forceCpu = False
# 使用CPU
if forceCpu:
    cpu = tf.config.list_physical_devices("CPU")
    tf.config.set_visible_devices(cpu)
    print(tf.config.list_logical_devices())

# 动态显存
if not forceCpu:
    physical_devices = tf.config.list_physical_devices('GPU')
    try:
        tf.config.experimental.set_memory_growth(physical_devices[0], True)
    except:
        print('Invalid device or cannot modify virtual devices once initialized')

plt.rcParams['font.sans-serif'] = ['SimHei']  # 黑体
plt.rcParams['axes.unicode_minus'] = False  # 解决无法显示符号的问题
palette = 'deep'
sns.set(font='SimHei', font_scale=1.2, style='whitegrid', palette=palette)  # 解决Seaborn中文显示问题

rawDataPivot = pd.read_excel('../算法评估/LSTM/LTSM预测值填充-物流网络历史货量数据.xlsx')

# # 宽表转长表
rawData = pd.melt(rawDataPivot, id_vars=['场地1', '场地2'], var_name='日期', value_name='货量')

D:\Users\win\AppData\Roaming\Python\Python310\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Invalid device or cannot modify virtual devices once initialized



KeyboardInterrupt



In [ ]:
rawDataPivot.head()

In [ ]:
rawData.head()

In [ ]:
def getxy():
    targetKey = '货量'
    # 对场地进行编码

    df = rawData.copy()
    # df = df[(df['场地1'] == 'DC14') & (df['场地2'] == 'DC10')].reset_index(drop=True)
    df['场地1'] = df['场地1'].str.replace('DC', '')
    df['场地1'] = df['场地1'].astype('int64')
    df['场地2'] = df['场地2'].str.replace('DC', '')
    df['场地2'] = df['场地2'].astype('int64')

    df['日期'] = pd.to_datetime(df['日期'])
    df['日期'] = df['日期'] - df['日期'].min()
    df['日期'] = df['日期'].apply(lambda x: x.days)

    df['货量'] = np.log(df['货量'])
    return df.drop(targetKey, axis=1), df[targetKey], df


def getPibotXy():
    # 对场地进行编码

    df = rawDataPivot.copy()
    df['场地1'] = df['场地1'].str.replace('DC', '')
    df['场地1'] = df['场地1'].astype('int64')
    df['场地2'] = df['场地2'].str.replace('DC', '')
    df['场地2'] = df['场地2'].astype('int64')

    df.iloc[:, 2:] = np.log(df.iloc[:, 2:])
    return df

In [ ]:
_, _, data = getxy()


In [ ]:
filterSize = 5

In [ ]:
modelPath = '../算法评估/LSTM/LSTM.1.h5'


# 定义自定义指标函数
def r_square(y_true, y_pred):
    SS_res = tf.keras.backend.sum(tf.keras.backend.square(y_true - y_pred))
    SS_tot = tf.keras.backend.sum(tf.keras.backend.square(y_true - tf.keras.backend.mean(y_true)))
    return (1 - SS_res / (SS_tot + tf.keras.backend.epsilon()))


model = tf.keras.models.load_model(modelPath, custom_objects={'r_square': r_square})
metrics = r_square
metricsName = metrics.__name__
model.compile(loss='mse', optimizer='adam', metrics=[metrics])

# # 测试模型
# y_pred = model.predict(trainX, batch_size=5000)
# y_pred = np.round(np.exp(y_pred))

In [ ]:
trainX = rawDataPivot.iloc[:, -4:]
trainX

In [ ]:
import datetime

rawDataPivotPredict = getPibotXy()
print(rawDataPivotPredict.dtypes)
rawDataPivotPredict.head()

In [ ]:
预测长度 = 30
for i in range(0, 预测长度 + 1):
    # 测试变量形状
    temp = pd.merge(rawDataPivotPredict.iloc[:, :2], rawDataPivotPredict.iloc[:, 0 - 4:], left_index=True,
                    right_index=True)
    temp = pd.melt(temp, id_vars=['场地1', '场地2'], var_name='日期', value_name='货量')

    temp['日期'] = pd.to_datetime(temp['日期'])
    temp['日期'] = temp['日期'] - rawData['日期'].min()
    temp['日期'] = temp['日期'].apply(lambda x: x.days)

    # print(temp.dtypes)
    # temp

    # ------------------------------

    # 准备数据

    # 按照时间排序
    data = temp.sort_values('日期')

    # 将每条有向边转换成一个序列
    dataset = []
    for _, group in data.groupby(['场地1', '场地2']):
        dataset.append(group.values)

    # print(f'dataset数量: {len(dataset)}')
    # print(dataset[0].shape)

    # ------------------------------

    look_back = filterSize - 1
    trainX = []
    for i in range(len(dataset)):
        train_x = np.reshape(dataset[i], (1, 4, 4))
        trainX.append(train_x)

    trainX = np.concatenate(trainX)
    trainX = trainX.astype('float64')
    # print(trainX.shape)
    # trainX

    # ------------------------------

    # 先预测后一天
    y_pred = model.predict(trainX, batch_size=1000)
    nextCol = rawDataPivotPredict.columns[-1] + datetime.timedelta(days=1)
    rawDataPivotPredict[nextCol] = y_pred
    # print(rawDataPivotPredict)
    # break

    # 再自回归
    model.fit(trainX, y_pred, epochs=200, batch_size=5000)


In [ ]:
rawDataPivotPredict.head()

In [ ]:
rawDataPivotPredict.iloc[:, 2:] = np.round(np.exp(rawDataPivotPredict.iloc[:, 2:]))

In [ ]:
rawDataPivotPredict

In [ ]:
print(rawDataPivotPredict.columns[4])
print(type(rawDataPivotPredict.columns[4]))
isinstance(rawDataPivotPredict.columns[4], datetime.datetime)

In [ ]:
for col in rawDataPivotPredict.columns:
    # 判断列是否为日期时间类型
    if isinstance(col, datetime.datetime):
        # 将日期时间转换为日期，并将结果存储回原来的列名
        new_col_name = col.strftime('%Y-%m-%d')
        print(new_col_name)
        rawDataPivotPredict = rawDataPivotPredict.rename(columns={col: new_col_name})
        rawDataPivotPredict[new_col_name] = rawDataPivotPredict[new_col_name].astype('int64')

rawDataPivotPredict.columns

In [ ]:
rawDataPivotPredict

In [ ]:
rawDataPivotPredict.to_excel('第一问结果LTSM-物流网络历史货量数据.xlsx', index=False)